<a href="https://colab.research.google.com/github/LM1997610/NN/blob/main/HW_1_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

import matplotlib.pyplot as plt

In [2]:
#a,b, c = tfds.load("stl10", split=["train", "test[1000:]", "test[1000:]"], as_supervised=True, batch_size = 300)

In [3]:
train_data, val_data, test_data = tfds.load("cifar10", split=["train[0:49000]", "train[49000:]", "test"], as_supervised=True, batch_size = 300)

In [4]:
def preproccess(image, label):
  image = tf.cast(image, tf.float32)/ 255.0
  return image, label

In [5]:
train_data= train_data.map(preproccess)
val_data= val_data.map(preproccess)
test_data= test_data.map(preproccess)

In [6]:
def add_conv_block(model, n_filters):

  model.add(keras.layers.Conv2D(filters = n_filters,                 # conv_layer
                                kernel_size = (3, 3),
                                padding = "same"))

  model.add(keras.layers.BatchNormalization())                       # batch_normalization
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))  # pool_layer
  model.add(keras.layers.Activation('relu'))                         # activation
  model.add(keras.layers.Dropout(0.3))                               # drop_out

  return model

In [9]:
hidden_layers =  [128, 256, 512, 512, 512]

model = keras.Sequential()

for i in hidden_layers:
  model = add_conv_block(model, i)

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10))


In [10]:
model.build(input_shape=(256, 32,32,3))
# model.summary()

In [11]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = keras.metrics.SparseCategoricalAccuracy()
opt = keras.optimizers.Adam(learning_rate=0.001, weight_decay=0.001)

In [12]:
callb = [tf.keras.callbacks.TerminateOnNaN(),
         tf.keras.callbacks.EarlyStopping( patience=5, restore_best_weights=True, verbose=1),
         tf.keras.callbacks.TensorBoard(log_dir="logs", update_freq = 50 )]

In [ ]:
model.compile(optimizer= opt,
              loss = cross_entropy,
              metrics = accuracy)

history = model.fit(train_data, epochs=20, validation_data = val_data, callbacks=callb )

Epoch 1/20
164/164 [==============================] - 36s 116ms/step - loss: 1.5735 - sparse_categorical_accuracy: 0.4171 - val_loss: 3.4197 - val_sparse_categorical_accuracy: 0.0920
Epoch 2/20
164/164 [==============================] - 17s 104ms/step - loss: 1.0820 - sparse_categorical_accuracy: 0.6112 - val_loss: 2.7146 - val_sparse_categorical_accuracy: 0.2600
Epoch 3/20
164/164 [==============================] - 17s 105ms/step - loss: 0.8947 - sparse_categorical_accuracy: 0.6845 - val_loss: 1.5791 - val_sparse_categorical_accuracy: 0.4450
Epoch 4/20
164/164 [==============================] - 18s 108ms/step - loss: 0.7712 - sparse_categorical_accuracy: 0.7287 - val_loss: 0.9359 - val_sparse_categorical_accuracy: 0.6510
Epoch 5/20
134/164 [=======================>......] - ETA: 3s - loss: 0.6993 - sparse_categorical_accuracy: 0.7554

In [ ]:
test_loss, test_acc = model.evaluate(c, verbose=2)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(11, 3))
fig.tight_layout(pad=4, w_pad = 6.5)

ax[0].plot(history.history['loss'], label='train_loss', color = "red")
ax[0].plot(history.history['val_loss'], label='val_loss', color = "green")

ax[0].set_title('Loss History\n', fontsize=14)
ax[0].set_xlabel('\n Epochs')
ax[0].set_ylabel('Loss')
#ax[0].set_xticks(list(range(1, num_epochs+1, 3)))
ax[0].grid(linewidth=0.4)
ax[0].legend(loc="best", prop={'size': 8})


ax[1].plot(history.history['sparse_categorical_accuracy'], label='train_accuracy', color = "red")
ax[1].plot(history.history['val_sparse_categorical_accuracy'], label = 'val_accuracy', color = "green")
ax[1].set_title('Classification Accuracy History\n', fontsize=14)
ax[1].set_xlabel('\n Epochs')
ax[1].set_ylabel('Classification accuracy')
ax[1].grid(linewidth=0.4)

#ax[1].set_xticks(list(range(1, num_epochs+1, 3)))
ax[1].legend(loc ="lower right", prop={'size': 8})

#plt.savefig("history_plot.png")
plt.show()

In [ ]:
cifar10_class_names = {0: "Plane", 1: "Car", 2: "Bird", 3: "Cat", 4: "Deer",
                       5: "Dog", 6: "Frog", 7: "Horse", 8: "Boat", 9: "Truck"}

In [ ]:
import numpy as np

In [ ]:
def show_stuff(index, model):

  map = silency_map(test_images[index], model)

  last = test_images[index].numpy()[:,:,1] + map

  fig, ax = plt.subplots(1, 3, figsize=(5, 5))

  ax[1].axis('off')

  ax[0].imshow(test_images[index]);
  ax[0].axis('off')

  ax[1].imshow(map,alpha=0.8, cmap="jet")

  ax[2].imshow(last)
  #fig.colorbar(i)

  plt.show()

In [ ]:
index = 0  # Indice del sample che vuoi estrarre
c_list = list(c)  # Converti il dataset in una lista

first_image, first_label = c_list[index]

In [ ]:
X_train = list(map(lambda x: x[0], c))
y_train = list(map(lambda x: x[1], c))

In [ ]:
X_train[0][0].numpy().shape

In [ ]:
def silency_map(image, model):

  with tf.GradientTape() as tape:

    tensor_input = tf.convert_to_tensor(np.expand_dims(image, axis=0))
    print(tensor_input.shape)

    tape.watch(tensor_input)

    class_scores = model(tensor_input, training=False)
    predicted_class = np.argmax(class_scores)
    class_channel = class_scores[:,predicted_class]

  grads = tape.gradient(class_channel, tensor_input)
  dgrad_abs = tf.math.abs(grads)
  dgrad_max_ = np.max(dgrad_abs, axis=3)[0]

  ## normalize to range between 0 and 1
  arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
  map = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)

  return map, grads

In [ ]:
X_train[0][0].shape

In [ ]:
map, grads = silency_map(X_train[0][0], model)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(14,5))

axes[0].imshow(test_images[2])
i = axes[1].imshow(a,cmap="jet",alpha=0.8)
fig.colorbar(i)
